In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

plt.style.use('ggplot')

In [ ]:
# Source data was initially adjusted to remove unnecessary comments and rename column names
df = pd.read_csv('input/Dataset_spine.csv', sep=',')

# Remove, if any, empty values from the dataset
df = df.dropna()

# Map classification attribute from string to binary
df["Class_att"] = df["Class_att"].map({"Normal": 0, "Abnormal": 1})

In [ ]:
# Separate data into training and validation
X = df.drop(["Class_att"], axis=1)
y = df["Class_att"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69)

In [ ]:
# MLP training
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', learning_rate='constant', solver='adam', max_iter=10000, tol=1e-4)

In [ ]:
mlp.fit(X_train, y_train)

print("Training set score: {:.4f}".format(mlp.score(X_train, y_train)))
print("Test set score: {:.4f}".format(mlp.score(X_test, y_test)))

In [ ]:
# Confusion matrix
y_pred = mlp.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

In [ ]:
normal_data = list()
abnormal_data = list()

pelvic_incidence = list()
pelvic_tilt = list()
lumbar_lordosis_angle = list()
sacral_slope = list()
pelvic_radius = list()
degree_spondylolisthesis = list()
pelvic_slope = list()
direct_tilt = list()
thoracic_slope = list()
cervical_tilt = list()
sacrum_angle = list()
scoliosis_slope = list()

for i in range(2):
    pelvic_incidence.append([])
    pelvic_tilt.append([])
    lumbar_lordosis_angle.append([])
    sacral_slope.append([])
    pelvic_radius.append([])
    degree_spondylolisthesis.append([])
    pelvic_slope.append([])
    direct_tilt.append([])
    thoracic_slope.append([])
    cervical_tilt.append([])
    sacrum_angle.append([])
    scoliosis_slope.append([])

hand = open('input/Dataset_spine.csv')
next(hand)
for line in hand:
    line = line.rstrip()
    vals = line.split(',')
    if vals[12] == 'Normal':
        pelvic_incidence[0].append(float(vals[0]))
        pelvic_tilt[0].append(float(vals[1]))
        lumbar_lordosis_angle[0].append(float(vals[2]))
        sacral_slope[0].append(float(vals[3]))
        pelvic_radius[0].append(float(vals[4]))
        degree_spondylolisthesis[0].append(float(vals[5]))
        pelvic_slope[0].append(float(vals[6]))
        direct_tilt[0].append(float(vals[7]))
        thoracic_slope[0].append(float(vals[8]))
        cervical_tilt[0].append(float(vals[9]))
        sacrum_angle[0].append(float(vals[10]))
        scoliosis_slope[0].append(float(vals[11]))
    else:
        pelvic_incidence[1].append(float(vals[0]))
        pelvic_tilt[1].append(float(vals[1]))
        lumbar_lordosis_angle[1].append(float(vals[2]))
        sacral_slope[1].append(float(vals[3]))
        pelvic_radius[1].append(float(vals[4]))
        degree_spondylolisthesis[1].append(float(vals[5]))
        pelvic_slope[1].append(float(vals[6]))
        direct_tilt[1].append(float(vals[7]))
        thoracic_slope[1].append(float(vals[8]))
        cervical_tilt[1].append(float(vals[9]))
        sacrum_angle[1].append(float(vals[10]))
        scoliosis_slope[1].append(float(vals[11]))

In [ ]:
figure, axis = plt.subplots(2, 3, figsize=(15, 10))

axis[0, 0].boxplot(pelvic_incidence, labels=['Normal', 'Abnormal'])
axis[0, 0].set_title("Pelvic Incidence")

axis[0, 1].boxplot(pelvic_tilt, labels=['Normal', 'Abnormal'])
axis[0, 1].set_title("Pelvic Tilt")

axis[0, 2].boxplot(lumbar_lordosis_angle, labels=['Normal', 'Abnormal'])
axis[0, 2].set_title("Lumbar Lordosis Angle")

axis[1, 0].boxplot(sacral_slope, labels=['Normal', 'Abnormal'])
axis[1, 0].set_title("Sacral Slope")

axis[1, 1].boxplot(pelvic_radius, labels=['Normal', 'Abnormal'])
axis[1, 1].set_title("Pelvic Radius")

axis[1, 2].boxplot(degree_spondylolisthesis, labels=['Normal', 'Abnormal'])
axis[1, 2].set_title("Degree Spondylolisthesis")

plt.show()

In [ ]:
figure, axis = plt.subplots(2, 3, figsize=(15, 10))

axis[0, 0].boxplot(pelvic_slope, labels=['Normal', 'Abnormal'])
axis[0, 0].set_title("Pelvic Slope")

axis[0, 1].boxplot(direct_tilt, labels=['Normal', 'Abnormal'])
axis[0, 1].set_title("Direct Tilt")

axis[0, 2].boxplot(thoracic_slope, labels=['Normal', 'Abnormal'])
axis[0, 2].set_title("Thoracic Slope")

axis[1, 0].boxplot(cervical_tilt, labels=['Normal', 'Abnormal'])
axis[1, 0].set_title("Cervical Tilt")

axis[1, 1].boxplot(sacrum_angle, labels=['Normal', 'Abnormal'])
axis[1, 1].set_title("Sacrum Angle")

axis[1, 2].boxplot(scoliosis_slope, labels=['Normal', 'Abnormal'])
axis[1, 2].set_title("Scoliosis Slope")

plt.show()